In [116]:
from openai import OpenAI
from secret import OPENAI_API_KEY
from PIL import Image
import base64
import os
import json
import re
import pandas as pd
import io

In [117]:
client = OpenAI(api_key=OPENAI_API_KEY)
model = "gpt-4o-mini"

In [119]:
def encode_image(image_path, max_size=(512, 512), quality=80):
    image = Image.open(image_path)

    # Redimensionner l'image
    image.thumbnail(max_size)

    # Convertir en bytes avec compression
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=quality)

    # Encoder en Base64
    encoded_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

    print(f"Encoded image: {image_path}")

    return encoded_string

In [120]:
def send_chat_request(message, extracting_json=False):
    try:
        response = client.chat.completions.create(
            model=model, messages=message
        )

        result = response.choices[0].message.content.strip()

        return extract_json(result)


    except Exception as e:
        print(f"Erreur OpenAI : {e}")

In [121]:
def chat_get_key_words(base64_images, image_names):

    # Liste pour chaque image et chaque texte associé
    content_list = []
    for i, base64_image in enumerate(base64_images):
        content_list.append({
            "type": "text",
            "text": f"""Décris moi toutes les images avec 5 mots-clés.
                        Retourne le résultat au format JSON comme ceci :
                        {{
                            {image_names[i]} : [mot-clé1, mot-clé2, mot-clé3, mot-clé4, mot-clé5]
                            ...

                        }}
                        """
        })
        content_list.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        })

    messages = [
        {
            "role": "user",
            "content": content_list
        }
    ]

    return send_chat_request(messages)


In [122]:
def extract_json(response_text):
    """
    Extrait la portion JSON (délimitée par {}) de la réponse textuelle pour seulement avoir le dictionnaire et non le texte généré par l'ia.
    """
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Erreur lors du chargement du JSON : {e}")
            return None
    else:
        print("Aucun JSON trouvé dans la réponse.")
        return None

In [123]:
def chat_get_categories(keywords_output):
    """
    Utilise les mots-clés extraits pour regrouper les images similaires en catégories.
    Les images sont identifiées par leur ordre dans la liste.
    """

    # Préparation d'un prompt détaillé incluant le résultat des mots-clés et l'ordre des images
    prompt = f"""Voici les listes de mots-clés obtenues pour chaque image (dans l'ordre) : {keywords_output}

    En te basant sur ces informations, regroupe les images similaires dans des catégories. Une catégorie est décrite par un seul mot-clé. Une image ne peut appartenir qu'à une seule catégorie.
    Retourne le résultat au format JSON comme tel :
    {{
        "categorie1": [ "name", "name" ],
        "categorie2": [ "name", "name" ],
        ...
    }}
    """
    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]

    return send_chat_request(messages)

In [124]:
directory = "test_data"
allowed_extensions = {".jpg", ".jpeg", ".png"}
image_paths = [
    os.path.join(directory, filename)
    for filename in os.listdir(directory)
    if os.path.splitext(filename)[1].lower() in allowed_extensions
]
base64_images = [encode_image(path) for path in image_paths]

Encoded image: test_data\20240826_184159.jpg
Encoded image: test_data\20250213_165625.jpg
Encoded image: test_data\20250217_081138.jpg
Encoded image: test_data\20250219_084504.jpg


In [125]:
image_names = [os.path.basename(path) for path in image_paths]
keywords_output = chat_get_key_words(base64_images, image_names)
print("Mots-clés par image :")
print(keywords_output)

Mots-clés par image :
{'20240826_184159.jpg': ['chemin', 'forêt', 'nature', 'feuilles', 'détente'], '20250213_165625.jpg': ['neige', 'tempête', 'route', 'visibilité', 'hiver'], '20250217_081138.jpg': ['neige', 'voiture', 'gel', 'hiver', 'paysage'], '20250219_084504.jpg': ['traffic', 'neige', 'autobus', 'hiver', 'route']}


9s -> 1 image

22s - 42s -> 4 images (pas constant)

In [126]:
categories_output = chat_get_categories(keywords_output)
print("Catégorisation des images :")
print(categories_output)

Catégorisation des images :
{'nature': ['20240826_184159.jpg'], 'hiver': ['20250213_165625.jpg', '20250217_081138.jpg', '20250219_084504.jpg']}


## Ajout des données au DataFrame

In [127]:
image_data = pd.read_csv("test_data.csv")
image_data

,Image,Path,Date,Place
0,20240826_184159.jpg,test_data\20240826_184159.jpg,2024:08:26 18:41:59,NaN
1,20250213_165625.jpg,test_data\20250213_165625.jpg,2025:02:13 16:56:25,NaN
2,20250217_081138.jpg,test_data\20250217_081138.jpg,2025:02:17 08:11:38,NaN
3,20250219_084504.jpg,test_data\20250219_084504.jpg,2025:02:19 08:45:04,"{1: 'N', 2: (48.0, 24.0, 43.5582), 3: 'W', 4: ..."


In [128]:
def add_keywords_to_df(image_data, keywords_output):
    if keywords_output:
        image_data["Keywords"] = image_data["Image"].apply(lambda img: keywords_output.get(img, None))
    return image_data

In [129]:
image_data = add_keywords_to_df(image_data, keywords_output)

image_data

,Image,Path,Date,Place,Keywords
0,20240826_184159.jpg,test_data\20240826_184159.jpg,2024:08:26 18:41:59,NaN,"[chemin, forêt, nature, feuilles, détente]"
1,20250213_165625.jpg,test_data\20250213_165625.jpg,2025:02:13 16:56:25,NaN,"[neige, tempête, route, visibilité, hiver]"
2,20250217_081138.jpg,test_data\20250217_081138.jpg,2025:02:17 08:11:38,NaN,"[neige, voiture, gel, hiver, paysage]"
3,20250219_084504.jpg,test_data\20250219_084504.jpg,2025:02:19 08:45:04,"{1: 'N', 2: (48.0, 24.0, 43.5582), 3: 'W', 4: ...","[traffic, neige, autobus, hiver, route]"


In [130]:
def add_categories_to_df(image_data, categories_output):
    if categories_output:
        image_data["Category"] = image_data["Image"].map(lambda image: next((cat for cat in categories_output if image in categories_output[cat]), None))
    else:
        print("Aucune catégorisation trouvée !")

    return image_data

In [131]:
image_data = add_keywords_to_df(image_data, keywords_output)
image_data = add_categories_to_df(image_data, categories_output)

In [132]:
image_data

,Image,Path,Date,Place,Keywords,Category
0,20240826_184159.jpg,test_data\20240826_184159.jpg,2024:08:26 18:41:59,NaN,"[chemin, forêt, nature, feuilles, détente]",nature
1,20250213_165625.jpg,test_data\20250213_165625.jpg,2025:02:13 16:56:25,NaN,"[neige, tempête, route, visibilité, hiver]",hiver
2,20250217_081138.jpg,test_data\20250217_081138.jpg,2025:02:17 08:11:38,NaN,"[neige, voiture, gel, hiver, paysage]",hiver
3,20250219_084504.jpg,test_data\20250219_084504.jpg,2025:02:19 08:45:04,"{1: 'N', 2: (48.0, 24.0, 43.5582), 3: 'W', 4: ...","[traffic, neige, autobus, hiver, route]",hiver
